# Correlation of Price and Sentiment Messages Number

## Intro

### Imports

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import os
import sys
from dotenv import load_dotenv


In [3]:
from typing import List, Union

import matplotlib.pyplot as plt
from  matplotlib.colors import LinearSegmentedColormap
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm, trange

from datetime import datetime as dt
from datetime import timedelta

from helixirapi.helixir_api import HelixirApi

In [4]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')

import plotly.io as pio
pio.renderers.default = "notebook_connected"

### Create client instance

In [5]:
load_dotenv()
AUTH_TOKEN = os.getenv("AUTH_TOKEN")
client = HelixirApi(auth_token=AUTH_TOKEN)
client

## Get Data

In [6]:
from_ = "2022-05-10"
to = "2022-06-01"
resolution = "D1"

client.get_token(contract="0xbA2aE424d960c26247Dd6c32edC70B295c744C43")

TokenResponse(
	id = 685,
	name = Dogecoin,
	symbol = DOGE,
	chain = BSC,
	active = True,
	decimals = 8.0,
	total_supply = 1400000000.0,
	circulating_supply = 1398977878.7484114,
	contract = 0xbA2aE424d960c26247Dd6c32edC70B295c744C43,
)

In [7]:
candles = pd.DataFrame([c.__dict__ for c in client.get_candles(
    contract = "0x3EE2200Efb3400fAbB9AacF31297cBdD1d435D47",
    from_ = from_,
    to = to,
    resolution = resolution,
)]).set_index("time")
candles

Iterating requests to meet the limit:   0%|          | 0/1 [00:00<?, ?it/s]

,open,high,low,close
time,,,,
2022-05-09 00:00:00+00:00,0.742159,0.752084,0.601878,0.601878
2022-05-10 00:00:00+00:00,0.601540,0.697380,0.583951,0.627792
2022-05-11 00:00:00+00:00,0.629235,0.659482,0.472842,0.512539
2022-05-12 00:00:00+00:00,0.512994,0.545571,0.376689,0.472623
2022-05-13 00:00:00+00:00,0.472629,0.604366,0.469098,0.527113
2022-05-14 00:00:00+00:00,0.526666,0.553432,0.490227,0.538704
2022-05-15 00:00:00+00:00,0.538450,0.597034,0.515621,0.596734
2022-05-16 00:00:00+00:00,0.596239,0.612645,0.546210,0.554959
2022-05-17 00:00:00+00:00,0.554937,0.597575,0.553454,0.575779


In [8]:
candles.shape, candles.index.nunique(), candles.drop_duplicates().shape

((21, 4), 21, (21, 4))

In [9]:
from dateutil.parser import isoparse

def get_messages_numbers(client, methods: List[str] = None, tag: str = "", candles: pd.DataFrame = None,
                        from_: Union[str, int, dt] = None, to: Union[str, int, dt] = None, resolution: str = None) -> pd.DataFrame:
    if not methods:
        methods = [
            "get_discord",
            "get_publications",
            "get_reddit",
            "get_telegram",
            "get_twitter",
        ]
    if candles is None:
        index = [i for i in range(int(isoparse(from_).timestamp()), int(isoparse(to).timestamp()), client.candle_seconds[resolution])]
        candles = pd.DataFrame(index=index)
    if not from_:
        from_ = candles.index[0]
    
    messages = pd.DataFrame(index=candles.index)   
    delta = candles.index[1] - candles.index[0]
    for method in methods:
        messages_counts = []
        m = []
        for t in tqdm(candles.index, leave=False, desc=f"Iterating time to retrieve {str(method).split('_')[-1]} messages of {tag} tag"):
            tt = m[-1].created_at if m else t
            if m and m[0].created_at > t:
                messages_number = 0
            else:
                above = len([mm for mm in m if mm.created_at > (t + delta)])
                messages_number = len(m) - above
                m = m[-above:]
            while tt < t + delta:
                new_m = getattr(client, method)(
                    from_ = tt,
                    limit = 400,
                    tag = tag,
                )
                above = len([mm for mm in new_m if mm.created_at > (t + delta)])
                m = new_m[-above:]
                new_m = new_m[:-above]
                messages_number += len(new_m)
                if new_m == [] or len(new_m) < 499:
                    tt = t + delta
                else:
                    tt = new_m[-1].created_at + timedelta(microseconds=1)
            messages_counts.append(messages_number)
        messages[str(method).split("_")[-1]] = messages_counts
    messages["total"] = messages.sum(axis=1)
    messages = messages.astype(int)
    return messages

messages = get_messages_numbers(
    client = client,
    tag = "doge",
    from_=from_,
    candles = candles,
)
messages

Iterating time to retrieve discord messages of doge tag:   0%|          | 0/21 [00:00<?, ?it/s]

Iterating time to retrieve publications messages of doge tag:   0%|          | 0/21 [00:00<?, ?it/s]

Iterating time to retrieve reddit messages of doge tag:   0%|          | 0/21 [00:00<?, ?it/s]

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

## Calculate Correlation

In [10]:
rwg_cmap=LinearSegmentedColormap.from_list('rg',["r", "w", "g"], N=256)
pd.DataFrame(
    messages.corrwith(candles["close"]),
    columns=["correlation"],
).T.style.background_gradient(
    cmap=rwg_cmap,
    axis=None,
    # vmin=-1,
    # vmax=1,
)

NameError: name 'messages' is not defined

## Bucket Processing

In [70]:
def calculate_price_messages_correlation(client, contract: str, symbol: str = None, chain: Union[str, int] = "bsc",
                                    from_: Union[str, int, dt] = None, to: Union[str, int, dt] = None, resolution: str = "H1",
                                    tag: str = None, methods: List[str] = None) -> pd.Series:
    if not symbol:
        token = client.get_token(contract=contract)
        symbol = token.symbol
    candles = pd.DataFrame([c.__dict__ for c in client.get_candles(
        symbol=symbol, contract=contract, from_=from_, to=to, chain=chain, resolution=resolution,
    )])
    if candles.shape[0] <= 1:
        print(f"No price data for {symbol} symbol ({contract}).")
        return
    candles = candles.set_index("time")
    messages = get_messages_numbers(
        client = client,
        methods=methods,
        tag = tag if tag else symbol,
        from_=from_,
        candles = candles,
    ).dropna(axis=0, how="all")
    return messages.corrwith(candles["close"])


In [15]:
tokens = pd.DataFrame([t.__dict__ for t in client.get_tokens(limit=42, extended=True)])
tokens

,id,name,symbol,chain,active,decimals,total_supply,circulating_supply,market_cap,liquidity_usd,price_usd,price_peg,price_change_24_h,price_change_7_d,volume_24_h,contract
0,10,Wrapped BNB,WBNB,BSC,True,18.0,4.872168e+06,4.872161e+06,1.130852e+09,7.133598e+20,2.253544e+02,1.000000e+00,-5.535,-20.232,6.174111e+08,0xbb4CdB9CBd36B01bD1cBaEBF2De08d9173bc095c
1,2,BUSD Token,BUSD,BSC,True,18.0,4.850999e+09,4.850774e+09,4.845429e+09,2.498289e+18,9.988980e-01,4.432588e-03,-0.378,-0.443,3.576921e+08,0xe9e7CEA3DedcA5984780Bafc599bD69ADd087D56
2,23,PancakeSwap Token,Cake,BSC,True,18.0,7.899806e+08,3.022353e+08,9.927590e+08,1.790681e+18,3.283777e+00,1.456959e-02,-7.176,-24.388,2.113425e+07,0x0E09FaBB73Bd3Ade0a17ECC321fD13a19e81cE82
3,6,Tether USD,USDT,BSC,True,18.0,4.979998e+09,4.979964e+09,4.973145e+09,1.521374e+16,9.986308e-01,4.431382e-03,0.006,-0.088,1.156232e+59,0x55d398326f99059fF775485246999027B3197955
4,694429,Bitcoin,XBTC,BSC,True,18.0,2.100000e+16,2.100000e+16,1.753158e+25,4.215631e+14,8.348370e+08,2.068397e+06,0.000,0.000,0.000000e+00,0x3E7B06B9c88ef97cF43C53F88F4f0689bDBd52AB
5,740948,Bitcoin,BTC,BSC,True,18.0,2.100000e+13,2.100000e+13,1.059938e+22,4.194548e+14,5.047326e+08,1.859682e+06,0.000,0.000,0.000000e+00,0x74042a20eb5F24cbE38AA1D5DaFc9bA6f07Eb35d
6,1012735,USDT,USDT,BSC,True,8.0,1.000000e+20,1.000000e+20,0.000000e+00,2.124138e+14,0.000000e+00,0.000000e+00,0.000,0.000,0.000000e+00,0x96e677F2bd7fFf30A0D6c25Be3f5820e6Be04CD3
7,699876,dead money token,DDM,BSC,True,18.0,1.000000e+13,1.000000e+13,4.942687e+16,5.078107e+12,4.942687e+03,1.175816e+01,0.000,0.000,0.000000e+00,0xe6A9C705165b27E5Ac2570A7C727921aAEa78bB8
8,699877,dead money token,DDM,BSC,True,18.0,1.000000e+13,1.000000e+13,4.978536e+16,5.041540e+12,4.978536e+03,1.184344e+01,0.000,0.000,0.000000e+00,0x37b53A16573DbFE9A1DcF5496C16E6C9d01b9618
9,1176,Super Floki,SLOKI,BSC,True,9.0,1.000000e+17,4.418478e+16,6.442461e+04,2.582861e+12,1.458037e-12,6.700000e-15,-6.514,-27.619,1.382861e+01,0xf7BF4507336AeA836ab5575eC1801d96c3059483


In [82]:
from_ = "2022-01-01"
to = "2022-06-01"
resolution = "D1"

correlations = pd.DataFrame()
for _, row in tqdm(tokens.iterrows(), total=tokens.shape[0], desc="Iteration of tokens"):
    c = calculate_price_messages_correlation(
        client=client,
        contract = row["contract"],
        symbol = row["symbol"],
        chain = row["chain"],
        from_ = from_,
        to = to,
        resolution = resolution,
    )
    if c is None or all(c.isna()):
        continue
    correlations[row["symbol"]] = c

correlations = correlations.T
rwg_cmap=LinearSegmentedColormap.from_list('rg',["r", "w", "g"], N=256)
correlations.style.background_gradient(
    cmap=rwg_cmap, 
    axis=None,
    vmin=-1,
    vmax=1,
).applymap(lambda x: 'background: black' if pd.isnull(x) else '')

Iteration of tokens:   0%|          | 0/42 [00:00<?, ?it/s]

Iterating requests to meet the limit:   0%|          | 0/6 [00:00<?, ?it/s]

Iterating time to retrieve discord messages of BTC tag:   0%|          | 0/2 [00:00<?, ?it/s]

Iterating time to retrieve publications messages of BTC tag:   0%|          | 0/2 [00:00<?, ?it/s]

Iterating time to retrieve reddit messages of BTC tag:   0%|          | 0/2 [00:00<?, ?it/s]

...

Iterating time to retrieve publications messages of AFP tag:   0%|          | 0/19 [00:00<?, ?it/s]

Iterating time to retrieve reddit messages of AFP tag:   0%|          | 0/19 [00:00<?, ?it/s]

Iterating time to retrieve telegram messages of AFP tag:   0%|          | 0/19 [00:00<?, ?it/s]

Iterating time to retrieve twitter messages of AFP tag:   0%|          | 0/19 [00:00<?, ?it/s]

,discord,publications,reddit,telegram,twitter,total
BTC,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
SLOKI,nan,nan,nan,-0.834155,-0.725535,-0.903720
GOLD NUGGET,-0.494408,nan,nan,nan,-0.391816,-0.580170
BTCB,-0.115337,nan,-0.281903,0.124603,0.252954,0.259307
USDC,0.165195,-0.008660,-0.133875,-0.011999,-0.085128,0.027236
RRB,nan,-0.818903,nan,-0.692849,-0.591757,-0.862761
FIST,0.085598,0.105567,0.170228,0.124320,0.098489,0.104555
OSK,nan,nan,nan,-0.026726,0.199256,0.188380
ETH,-0.411853,-0.074121,0.022706,-0.155994,-0.155994,-0.372862
CRUSH,nan,nan,nan,-1.000000,nan,-1.000000


In [98]:
from  matplotlib.colors import LinearSegmentedColormap
rwg_cmap=LinearSegmentedColormap.from_list('rg',["r", "w", "g"], N=256)

correlations.round(4).replace(-1.0, np.nan).replace(1.0, np.nan).dropna(how="all").style.background_gradient(
    cmap=rwg_cmap, 
    axis=None,
    vmin=-1,
    vmax=1,
).applymap(lambda x: 'background: black' if pd.isnull(x) else '')

,discord,publications,reddit,telegram,twitter,total
SLOKI,nan,nan,nan,-0.834200,-0.725500,-0.903700
GOLD NUGGET,-0.494400,nan,nan,nan,-0.391800,-0.580200
BTCB,-0.115300,nan,-0.281900,0.124600,0.253000,0.259300
USDC,0.165200,-0.008700,-0.133900,-0.012000,-0.085100,0.027200
RRB,nan,-0.818900,nan,-0.692800,-0.591800,-0.862800
FIST,0.085600,0.105600,0.170200,0.124300,0.098500,0.104600
OSK,nan,nan,nan,-0.026700,0.199300,0.188400
ETH,-0.411900,-0.074100,0.022700,-0.156000,-0.156000,-0.372900
FON,nan,nan,nan,-0.125100,-0.125100,-0.125100
VALK,nan,-0.508100,nan,-0.184500,-0.094800,-0.109600
